## Training NN (ok, ok linear regression)

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
sample_size = 1000000
max_num = 1000

In [3]:
fib_data = pd.DataFrame(
    {'x1': np.random.rand(sample_size), 
     'x2': np.random.rand(sample_size)})
fib_data['y'] = fib_data['x1'] + fib_data['x2']

In [4]:
fib_data.head()

,x1,x2,y
0,0.916182,0.538276,1.454459
1,0.348541,0.432702,0.781243
2,0.639932,0.457578,1.097510
3,0.774024,0.590979,1.365003
4,0.876003,0.978154,1.854158


## Let's train it & vizualize  :)

In [5]:
## Network configuration
learning_rates = [1, 0.1, 0.01, 0.001]

with tf.name_scope("input_data"):
    input_numbers = tf.placeholder(tf.float32, shape=[None, 2], name="input")
    output_numbers = tf.placeholder(tf.float32, shape=[None, 1], name="output")
    learning_rate = tf.placeholder(tf.float32)

with tf.name_scope("layer"):
    weight = tf.Variable(tf.truncated_normal([2, 1]), dtype=tf.float32, name="weight")
    bias = tf.Variable(tf.zeros([1]), dtype=tf.float32, name="bias")
    prediction = tf.matmul(input_numbers, weight) + bias 
    tf.summary.histogram('weight', weight) ###
    tf.summary.histogram('bias', bias)
    tf.summary.histogram('prediction', prediction)
    
    
with tf.name_scope("evaluation"):
    loss = tf.reduce_mean(tf.square(prediction - output_numbers), name="loss")
    tf.summary.scalar('loss', loss) ###
    

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train = optimizer.minimize(loss)

In [6]:
epochs = 500
feed_dict = {input_numbers: fib_data[['x1', 'x2']], output_numbers: fib_data[['y']]}
init = tf.global_variables_initializer()

In [7]:
with tf.Session() as sess:
    for lr in learning_rates:
        print('learning rate', lr)
        feed_dict[learning_rate] = lr
        sess.run(init)    
        merged_summary = tf.summary.merge_all()
        writer = tf.summary.FileWriter("logs/02/"+str(lr)) ###
        writer.add_graph(sess.graph) ###
    
        for epoch in range(epochs):
            if epoch % 50 == 0:
                print('loss', sess.run(loss, feed_dict))
            if epoch != 0 and epoch % 10 == 0:
                loss_so_far = sess.run(loss, feed_dict)
                if np.isnan(loss_so_far):
                    break
                s = sess.run(merged_summary, feed_dict)
                writer.add_summary(s, epoch) 
            sess.run(train, feed_dict)
        a = sess.run(prediction, feed_dict)

learning rate 1
loss 3.8946
loss 7.81696e+31
loss inf
learning rate 0.1
loss 0.0699729
loss 0.00935145
loss 0.00180536
loss 0.000357867
loss 7.39483e-05
loss 1.62251e-05
loss 3.84445e-06
loss 9.91535e-07
loss 2.76951e-07
loss 8.24552e-08
learning rate 0.01
loss 1.68316
loss 0.170503
loss 0.0924049
loss 0.0797345
loss 0.0710079
loss 0.0633551
loss 0.0565455
loss 0.0504802
loss 0.0450764
loss 0.0402602
learning rate 0.001
loss 3.6649
loss 2.76359
loss 2.09926
loss 1.60942
loss 1.24802
loss 0.981234
loss 0.784067
loss 0.638198
loss 0.530093
loss 0.449788


`tensorboard --logdir=logs/02/'